In [1]:
import numpy as np
from numpy import float32
import warnings
import os
import glob
import tensorflow as tf
import cv2
from sklearn import utils
from sklearn import preprocessing
import matplotlib.pyplot as plt

from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.applications import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing import *
from tensorflow.keras.utils import *
from tensorflow.keras import layers

from utils import *
from data_augmentation import *
from feature_extractor import *

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
IMG_SIZE=128

In [4]:
TRAIN_DATA_DIR = './lstm_data/train'
VAL_DATA_DIR = './lstm_data/val'
TEST_DATA_DIR = './lstm_data/test'

# AUG_TRAIN_DATA_DIR = './lstm_data_augmented/train'
# AUG_VAL_DATA_DIR = './lstm_data_augmented/val'
# AUG_TEST_DATA_DIR = './lstm_data_augmented/test'

In [5]:
# for subdir, dirs, files in os.walk(AUG_TRAIN_DATA_DIR):
#     for file in files:
#         filepath = os.path.join(subdir, file)
#         img = np.load(filepath)
#         # rotated_imgs = [rotate_img(img, 90), rotate_img(img, 180), rotate_img(img, 270)]
#         # for idx, rot_img in enumerate(rotated_imgs):
#         #     rotated_img_save = os.path.join(subdir, f'{file}_{idx}_rot')
#         #     np.save(rotated_img_save, rot_img)
            
#         fliplr_img = np.fliplr(img)
#         flipud_img = np.flipud(img)
        
#         fliplr_img_save = os.path.join(subdir, f'{file}_fliplr')
#         flipud_img_save = os.path.join(subdir, f'{file}_flipud')
        
#         np.save(fliplr_img_save, fliplr_img)
#         np.save(flipud_img_save, flipud_img)
        
#         # translated_img = translate(img)
#         # translated_img_save = os.path.join(subdir, f'{file}_trans')
#         # np.save(translated_img_save, translated_img)

In [62]:
class CustomDataGen(tf.keras.utils.Sequence):
    
    def __init__(self, folder_paths,
                 batch_size,
                 input_size=(6, 128, 128),
                 shuffle=True):
        
        self.folder_paths = folder_paths.copy()
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        
        self.n = len(self.folder_paths)
        self.n_category = 2
        # self.n_name = df[y_col['name']].nunique()
        # self.n_type = df[y_col['type']].nunique()
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.folder_paths)
    
    def __getitem__(self, index):
        batches = self.folder_paths[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__get_data(batches)  
        X = np.expand_dims(X, axis=4)
        return X, y
    
    def __len__(self):
        return self.n // self.batch_size
    
    def __get_input(self, folder):
        images = []
        for subdir, dirs, files in os.walk(folder):
            for f in files:
                images.append(os.path.join(subdir, f.decode(encoding='UTF-8')))
        images = sorted(images)
        images = [preprocessing.normalize(np.load(x)) for x in images]
        if len(images) != 6:
            print(folder)
        images = [cv2.resize(x, (128, 128), interpolation = cv2.INTER_AREA) for x in images]
        return np.array(images)

    def __get_output(self, path, num_classes=2):
        label = None
        folder = path.rsplit('/')[-2]
        if folder == 'positive':
            label = 1
        elif folder == 'negative':
            label = 0
        return tf.keras.utils.to_categorical(label, num_classes=num_classes)
    
    def __get_data(self, batches):
        # Generates data containing batch_size samples
        # path_batch = batches[self.X_col['path']]
        # category_batch = batches[self.y_col['type']]
        # print(batches)
        X_batch = np.asarray([self.__get_input(x) for x in batches])
        y_batch = np.asarray([self.__get_output(y, self.n_category) for y in batches])

        return X_batch, y_batch

In [63]:
train_folders = []
for subdir, dirs, files in os.walk(TRAIN_DATA_DIR):
     for d in dirs:
            if d != 'positive' and d != 'negative':
                train_folders.append(os.path.join(subdir, d))
train_folders = np.array(train_folders)

val_folders = []
for subdir, dirs, files in os.walk(VAL_DATA_DIR):
     for d in dirs:
            if d != 'positive' and d != 'negative':
                val_folders.append(os.path.join(subdir, d))
val_folders = np.array(val_folders)

test_folders = []
for subdir, dirs, files in os.walk(TEST_DATA_DIR):
     for d in dirs:
            if d != 'positive' and d != 'negative':
                test_folders.append(os.path.join(subdir, d))
test_folders = np.array(test_folders)

In [64]:
# def build_feature_extractor():
#     feature_extractor = CustomResNet50(
#         weights=None,
#         include_top=False,
#         input_shape=(IMG_SIZE, IMG_SIZE, 1),
#     )

#     inputs = keras.Input((IMG_SIZE, IMG_SIZE, 1))
#     # preprocessed = preprocess_input(inputs)

#     outputs = feature_extractor(inputs)
#     return Model(inputs, outputs, name="feature_extractor")


# feature_extractor = build_feature_extractor()

In [65]:
# def prepare_all_videos(df, root_dir):
#     num_samples = len(df)
#     video_paths = df["video_name"].values.tolist()
#     labels = df["tag"].values
#     labels = label_processor(labels[..., None]).numpy()

#     # `frame_masks` and `frame_features` are what we will feed to our sequence model.
#     # `frame_masks` will contain a bunch of booleans denoting if a timestep is
#     # masked with padding or not.
#     frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
#     frame_features = np.zeros(
#         shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
#     )

#     # For each video.
#     for idx, path in enumerate(video_paths):
#         # Gather all its frames and add a batch dimension.
#         frames = load_video(os.path.join(root_dir, path))
#         frames = frames[None, ...]

#         # Initialize placeholders to store the masks and features of the current video.
#         temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
#         temp_frame_features = np.zeros(
#             shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
#         )

#         # Extract features from the frames of the current video.
#         for i, batch in enumerate(frames):
#             video_length = batch.shape[0]
#             length = min(MAX_SEQ_LENGTH, video_length)
#             for j in range(length):
#                 temp_frame_features[i, j, :] = feature_extractor.predict(
#                     batch[None, j, :]
#                 )
#             temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

#         frame_features[idx,] = temp_frame_features.squeeze()
#         frame_masks[idx,] = temp_frame_mask.squeeze()

#     return (frame_features, frame_masks), labels


# train_data, train_labels = prepare_all_videos(train_df, "train")
# test_data, test_labels = prepare_all_videos(test_df, "test")

# print(f"Frame features in train set: {train_data[0].shape}")
# print(f"Frame masks in train set: {train_data[1].shape}")

In [66]:
# train = []

# for sample in traingen:
#     new_batch = []
#     batch = sample[0]
#     for b in batch:
#         tmp_frame_features = []
#         for frame in b:
#             print(frame.shape)
#             tmp_frame_features = feature_extractor.predict(frame)
#         new_batch.append(tmp_frame_features)
#     train.append(np.array(new_batch))

# train = np.array(train)

In [67]:
epochs = 20
batch_size = 8

In [68]:
traingen = CustomDataGen(train_folders, batch_size)
valgen = CustomDataGen(val_folders, batch_size)

In [69]:
#### Construct the input layer with no definite frame size.
inp = layers.Input(shape=(6, 128, 128, 1))

# We will construct 3 `ConvLSTM2D` layers with batch normalization,
# followed by a `Conv3D` layer for the spatiotemporal outputs.
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(5, 5),
    padding="same",
    return_sequences=True,
    activation="relu",
)(inp)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(3, 3),
    padding="same",
    return_sequences=True,
    activation="relu",
)(x)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(1, 1),
    padding="same",
    return_sequences=True,
    activation="relu",
)(x)
# x = layers.Conv3D(
#     filters=1, kernel_size=(3, 3, 3), activation="sigmoid", padding="same"
# )(x)
x = Dense(128, activation='relu')(x)
x = layers.GlobalAveragePooling3D()(x)
x = Dense(2, activation='softmax')(x)
# x = layers.Dense(1, activation='sigmoid')(x)

# Next, we will build the complete model and compile it.
model = Model(inp, x)

In [70]:
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 6, 128, 128, 1)]  0         
                                                                 
 conv_lstm2d_15 (ConvLSTM2D)  (None, 6, 128, 128, 64)  416256    
                                                                 
 batch_normalization_10 (Bat  (None, 6, 128, 128, 64)  256       
 chNormalization)                                                
                                                                 
 conv_lstm2d_16 (ConvLSTM2D)  (None, 6, 128, 128, 64)  295168    
                                                                 
 batch_normalization_11 (Bat  (None, 6, 128, 128, 64)  256       
 chNormalization)                                                
                                                                 
 conv_lstm2d_17 (ConvLSTM2D)  (None, 6, 128, 128, 64)  3302

In [71]:
# t = traingen[np.random.randint(0, 209)][0][np.random.randint(0, 8)]
# # Construct a figure on which we will visualize the images.
# fig, axes = plt.subplots(2, 3, figsize=(10, 8))

# # Plot each of the sequential images for one random data example.
# # data_choice = np.random.choice(range(len(x_train)), size=1)[0]
# for idx, ax in enumerate(axes.flat):
#     ax.imshow(t[idx])
#     ax.set_title(f"Frame {idx + 1}")
#     ax.axis("off")

# # Print information and display the figure.
# plt.show()

In [72]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(traingen, validation_data = valgen, epochs=epochs, verbose=2)

Epoch 1/20


2022-07-20 15:10:14.417486: E tensorflow/stream_executor/cuda/cuda_blas.cc:232] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2022-07-20 15:10:14.417533: E tensorflow/stream_executor/cuda/cuda_blas.cc:234] Failure to initialize cublas may be due to OOM (cublas needs some free memory when you initialize it, and your deep-learning framework may have preallocated more than its fair share), or may be because this binary was not built with support for the GPU in your machine.
2022-07-20 15:10:14.417638: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at conv_ops.cc:1120 : NOT_FOUND: No algorithm worked!  Error messages:
  Profiling failure on CUDNN engine 1#TC: UNKNOWN: CUDNN_STATUS_EXECUTION_FAILED
in tensorflow/stream_executor/cuda/cuda_dnn.cc(4023): 'cudnnConvolutionForward( cudnn.handle(), alpha, input_nd_.handle(), input_data.opaque(), filter_.handle(), filter_data.opaque(), conv_.handle(), ToConvForwardAlgo(algo), scratch_memory.opaque(), scratch_memo

NotFoundError: Graph execution error:

Detected at node 'model_5/conv_lstm2d_15/convolution' defined at (most recent call last):
    File "/home/master/09/nikitagalayda/miniconda3/envs/mobilenet_v2_test/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/master/09/nikitagalayda/miniconda3/envs/mobilenet_v2_test/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/master/09/nikitagalayda/miniconda3/envs/mobilenet_v2_test/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/master/09/nikitagalayda/miniconda3/envs/mobilenet_v2_test/lib/python3.9/site-packages/traitlets/config/application.py", line 976, in launch_instance
      app.start()
    File "/home/master/09/nikitagalayda/miniconda3/envs/mobilenet_v2_test/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    

In [ ]:
testgen = CustomDataGen(test_folders, batch_size)
predictions = model.predict(testgen)

In [ ]:
predictions

In [ ]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
  
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x

In [ ]:
def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
   
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

In [ ]:
def ResNet50(include_top=True, weights=None,
             input_tensor=None, input_shape=None,
             pooling=None,
             classes=2):
    """Instantiates the ResNet50 architecture.
    Optionally loads weights pre-trained
    on ImageNet. Note that when using TensorFlow,
    for best performance you should set
    `image_data_format="channels_last"` in your Keras config
    at ~/.keras/keras.json.
    The model and the weights are compatible with both
    TensorFlow and Theano. The data format
    convention used by the model is the one
    specified in your Keras config file.
    # Arguments
        include_top: whether to include the fully-connected
            layer at the top of the network.
        weights: one of `None` (random initialization)
            or "imagenet" (pre-training on ImageNet).
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 244)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 197.
            E.g. `(200, 200, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    #if weights == 'imagenet' and include_top and classes != 15:
    #    raise ValueError('If using `weights` as imagenet with `include_top`'
    #                     ' as true, `classes` should be 15')

    # Determine proper input shape
    
    # input_shape = _obtain_input_shape(input_shape,
    #                                   default_size=256,
    #                                   min_size=197,
    #                                   data_format=K.image_data_format(),
    #                                   require_flatten=include_top)
    
    input_shape = (128, 128, 1)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
            
    print(K.image_data_format())
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1

    x = ZeroPadding2D((3, 3))(img_input)
    x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    x = AveragePooling2D((4, 4), name='avg_pool')(x)

#     # x = Flatten()(x)
#     # x = Dense(1, activation='sigmoid', name='fc2')(x)
    
    if include_top:
        x = Flatten()(x)
        x = Dense(2, activation='softmax', name='fc2')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    # x = GlobalAveragePooling2D()(x)
    # x = Dense(2, activation = 'softmax')(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='resnet50')

    return model

In [ ]:
epochs = 100
batch_size = 32

model = ResNet50(include_top=True)

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(traingen, validation_data = valgen, epochs=epochs)

In [ ]:
# test_x_pos = get_data_array(f'{TEST_DATA_DIR}/positive')
# test_x_neg = get_data_array(f'{TEST_DATA_DIR}/negative')
# test_x = np.append(test_x_pos, test_x_neg, axis=0)

# test_y_pos = np.ones((test_x_pos.shape[0]), dtype=int)

# test_y_neg = np.zeros((test_x_neg.shape[0]), dtype=int)
# test_y = np.append(test_y_pos, test_y_neg, axis=0)

In [ ]:
test_data = []
test_data_y = []
for f in test_files:
    label = f.split('/')[-2]
    num_label = 0
    if label == 'positive':
        num_label = 1
    hot_enc_label = np.zeros(2)
    hot_enc_label[num_label] = 1
    test_data_y.append(hot_enc_label)
    image = np.load(f)
    image = cv2.resize(image, (128, 128), interpolation = cv2.INTER_AREA)
    image.reshape(128, 128, 1)
    test_data.append(image)
test_data = np.array(test_data)
test_data_y = np.array(test_data_y)

In [ ]:
predictions = model.predict(test_data)

In [ ]:
preds = []
for idx, p in enumerate(predictions):
    p_largest = p.argmax()
    pred = np.zeros(2)
    pred[p_largest] = 1
    preds.append(pred)